In [ ]:
import sys
import json

general = {'step_size':'0.001', 'use_road_network':True, 'catchment_radius': '500'}
# catchment_radius={'bus':'500','subway':'800','rail':'1000'}

params = {
    'general': general,
    # 'catchment_radius':catchment_radius
    }

default = {'training_folder': '../../scenarios/clermont', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/clermont', 'params': {'general': {'step_size': '0.001', 'use_road_network': True}, 'catchment_radius': {'bus': '500', 'subway': '800', 'rail': '1000'}}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from typing import Literal
import numba as nb
from collections import defaultdict
#num_cores = 1
print('numba threads',nb.config.NUMBA_NUM_THREADS)

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' 

numba threads 20


In [3]:
sys.path.insert(0, r'../../') # Add path
from utils import get_epsg, population_to_mesh, get_acf_distances, get_routing_distances

# Folders stucture and params

everything is on S3 (nothing on ECR) so no direct input folder. just scenarios/{scen}/inputs/

In [4]:
argv['training_folder']

'../../scenarios/clermont'

In [5]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/') ## réseau ferré
od_folder =  os.path.join(input_folder,'od/')

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

model_folder = os.path.join(input_folder, 'model/')

In [6]:
#zonage_input = gpd.read_file(input_folder + 'communes_zonage_serm.gpkg')
#zonage_input.rename(columns={'Population': 'population'}, inplace=True)
#zonage_input['density'] = zonage_input['population']/zonage_input.to_crs(epsg='4326').area/10**6
#zonage_input = zonage_input[['nom', 'insee_com', 'insee_dep', 'insee_reg', 'population', 'density', 'geometry']]
#zonage_input.to_file(input_folder + 'zonage.geojson')

In [7]:
input_catchment_radius = argv['params']['catchment_radius']

input_catchment_radius = {k:float(v) for k,v in catchment_radius.items()}
default_catchment_radius = 500

In [8]:
input_catchment_radius

{'bus': 500.0, 'subway': 800.0, 'rail': 1000.0}

In [9]:
catchment_radius

{'bus': '500', 'subway': '800', 'rail': '1000'}

In [10]:
step_size_min = 0.0005
step_size = max(float(argv['params']['general'].get('step_size')), step_size_min)
use_road_network = argv['params']['general'].get('use_road_network') ## param use r_ntw

In [11]:
step_size

0.001

In [12]:
use_road_network

True

# inputs

In [13]:
links = gpd.read_file(pt_folder + 'links.geojson',engine=io_engine) 
nodes = gpd.read_file(pt_folder + 'nodes.geojson',engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

In [14]:
nodes

,stop_code,stop_name,route_width,geometry
index,,,,
node_4b2VnuC1zTsPzd9ueovVuH,None,Thiers,3,POINT (3.50444 45.85987)
node_a3ZLvXzFnMqGNGu4hjEYT6,None,Ornon,3,POINT (3.38307 45.84869)
node_rwwVbaGFJtCHoKTZ6B83Ax,None,Clermont,3,POINT (3.09960 45.77888)
node_f8Fb3oREhTRCmTVNW7TQzE,None,Chatel Guyon,3,POINT (3.05999 45.92018)
node_b8e2FB6qm9PUQTLvQvNucR,None,Riom,3,POINT (3.07285 45.89581)
node_wbHMs2n7zPVS1pp1cXnNw5,None,Clermont,3,POINT (3.10296 45.78059)
node_2zTroDrJuDCKmipdvjBrtL,None,Gannat,3,POINT (3.20420 46.09740)
node_6CErzt5T6gFonN6RkpvAM6,None,Vichy,3,POINT (3.41873 46.12042)


In [15]:
#links['catchment_radius_marche'] = 500
#links['catchment_radius_velo'] = 1000
#links['catchment_radius_voiture'] = 4000

In [16]:
default_capacity = 60
if 'capacity' not in links.columns:
    links['capacity'] = default_capacity

In [17]:
catchment_radii = [x for x in links.columns if 'catchment_radius' in x]
catchment_radii_provided = (len(catchment_radii) > 0)
if len(catchment_radii) != 0:
    for x in catchment_radii:
        M = links[x].max()
        if links[x] != links[x].fillna(M):
            print('!! Catchemnt radius values missing in column {} !!'.format(x))
        links[x] = links[x].fillna(M)

In [18]:
# find meters CRS
centroid = [*LineString(nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

C:\Users\mlaune\AppData\Local\Temp\ipykernel_24652\550055817.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = [*LineString(nodes.centroid.values).centroid.coords][0]


32631

In [19]:
zonage_file = os.path.join(input_folder, 'zonage.geojson')
zonage_file_provided = os.path.isfile(zonage_file)
if zonage_file_provided :
    zonage = gpd.read_file(input_folder + 'zonage.geojson', engine=io_engine).to_crs(epsg='4326')
    #if 'population' not in zonage.columns:
    #    print('no population column defined')
    #if 'jobs' not in zonage.columns:
    #    print('no jobs column defined')
    #if 'tourists' not in zonage.columns:
    #    print('no tourists column defined')
    #if zonage.crs != 'EPSG:4326':
    #    zonage.to_crs(epsg='4326', inplace=True)
    #    print('CRS was set to 4326')
    zonage['area (km2)'] = zonage.to_crs(crs).area / 10**6
else:
    print('No zonage file in the input folder...')

In [20]:
zonage['density_patate'] = zonage['density']*3

In [21]:
densities = [x for x in zonage.columns if 'density' in x]

In [22]:
#population_provided = ('population' in zonage.columns)
#jobs_provided = ('jobs' in zonage.columns)
#tourists_provided = ('tourists' in zonage.columns)

In [23]:
#base_columns = [x for x in list(zonage.columns) if ((x not in ['population', 'jobs', 'tourists']) and ('density' not in x))]

#if population_provided:
#    population_file = zonage[base_columns + ['population']]
#    population_file['density'] = population_file['population'] / population_file['area (km2)']

#if jobs_provided:
#    jobs_file = zonage[base_columns + ['jobs']]
#    jobs_file.rename(columns={'jobs': 'population'}, inplace=True)
#    jobs_file['density'] = jobs_file['jobs'] / jobs_file['area (km2)']

#if tourists_provided:
#    tourists_file = zonage[base_columns + ['tourists']]
#    tourists_file.rename(columns={'tourists': 'population'}, inplace=True)
#    tourists_file['density'] = tourists_file['population'] / tourists_file['area (km2)']

In [24]:
#population_file = os.path.join(input_folder, 'population.geojson')
#population_file_provided = os.path.isfile(population_file)
#if population_file_provided :
#    population = gpd.read_file(population_file, engine=io_engine)
#    if 'index' in population.columns:
#        population = population.set_index('index')
#    else:
#        population.index.name='index'
#    assert 'density' in population.columns, 'need density column. in km2'
#    assert population.crs == 4326, 'population.geojson CRS must be EPSG:4326'
#print('population?',population_file_provided)

# Formation :
 ajouter un paramètre pour utiliser le réseau routier ou non. <br>
 Dans le modèle actuelle, le réseau routier est automatiquement utilisé pour :
 * faire un mesh de la population (affecter les gens aux noeds du réseau)
 * évaluer le stop catchment (la distance de routing est utilisé pour le catchment_radius)

 On pourrait ne pas vouloir utiliser le réseau meme si il est dans le projet! <br>
 TODO: changer rnodes_file_provided pour une combinaison entre un parametre et l'existance du fichier

In [25]:
## road network here
rnodes_file = os.path.join(road_folder, 'road_nodes.geojson')
rnodes_file_provided = os.path.isfile(rnodes_file)
if rnodes_file_provided:
    rnodes = gpd.read_file(os.path.join(road_folder, 'road_nodes.geojson'), engine=io_engine)
    rnodes = rnodes.set_index('index').to_crs(epsg='4326')
    rlinks = gpd.read_file(os.path.join(road_folder, 'road_links.geojson'), engine=io_engine)
    rlinks = rlinks.set_index('index').to_crs(epsg='4326')
print('rnodes?',rnodes_file_provided)

rnodes? True


In [26]:
od_file = os.path.join(od_folder, 'od.geojson')
od_file_provided = os.path.isfile(od_file)
if od_file_provided:
    od_test = gpd.read_file(od_file, engine=io_engine)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index']
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))
print('od?',od_file_provided)

od? False


In [27]:
# init results dfs
df_route_id = pd.DataFrame(index=links['route_id'].unique())
df_route_id.index.name = 'route_id'
df_route_id = df_route_id.reset_index()
df_route_id = df_route_id.merge(links[['route_id', 'route_type', 'capacity', 'headway']], on='route_id', how='left')
df_route_id = df_route_id.rename(columns={'capacity': 'veh_capacity (PAX)'})
df_route_id = df_route_id.drop_duplicates()
df_route_id = df_route_id.set_index('route_id')

df_route_type = pd.DataFrame(index=links['route_type'].unique())
df_route_type.index.name='route_type'

In [28]:
test = links.copy()[['frequency', 'headway']]

In [29]:
test['testtest'] = test['frequency']*test['headway']

In [30]:
test['freq_par_jour'] = test['frequency']*24*3600

In [31]:
test

,frequency,headway,testtest,freq_par_jour
index,,,,
link_qTNjaUDLm9x3yJrS9ubX9T,0.001667,600,1.0,144.0
link_wvPcELKEW52njEnuK7AeSe,0.001667,600,1.0,144.0
link_vTbX3V9qJJxThakJ1WZbbk,0.001667,600,1.0,144.0
link_8EachMgBpjP7T2bWPiAc3E,0.001667,600,1.0,144.0
link_purLhqauca5SLLCFkShEC7,0.001667,600,1.0,144.0
link_4hsYkThjafc1NVpc8kLeid,0.001667,600,1.0,144.0
link_1ck6aSDmpEy7Jo2fbhNAxE,0.001667,600,1.0,144.0
link_2sfH8EPaWDjhzsZiZJMz1d,0.001667,600,1.0,144.0
link_oCfgaw42pToYChhHThwCwy,0.001667,600,1.0,144.0


In [32]:
# Make sure headway are consistent : one single headway for both way and return
# Otherwise can't calculate KPIs later

df_route_id = df_route_id[~df_route_id.index.duplicated(keep='first')]
route_headway = dict(zip(df_route_id.index, df_route_id['headway']))
links.headway = links.route_id.map(route_headway)

In [33]:
def get_catchment_by_mode(col='route_id', pop_col='population', node_dist=None):
    #get all nodes with col filter
    link = links.groupby(col)[['a','b','route_type']].agg({'a':set,'b':set,'route_type':'first'})
    link['node'] = link.apply(lambda row: row['a'].union(row['b']), axis=1)
    link = link.drop(columns=['a','b'])
    # add catchment radius for the route_type
    link['catchment_radius'] = link['route_type'].apply(lambda x: input_catchment_radius.get(x,default_catchment_radius))

    col_exist = col == 'route_type' # cannot explode if index == route_type (a column)
    link = link.explode('node').reset_index(drop=col_exist)
    link = node_dist.merge(link, left_on='node_index', right_on='node')
    #filter by distance
    link = link[link['distances'] <= link['catchment_radius']]
    #drop duplicated mesh nodes (we count only one time)
    link = link.drop_duplicates(subset=['mesh_index',col],keep='first')

    return link.groupby(col)[pop_col].sum().to_dict()

In [34]:
def get_catchment_by_access(col='route_id', pop_col='population', catchment_col='catchment_radius', node_dist=None):
    #get all nodes with col filter
    link = links.groupby(col)[['a','b','route_type', catchment_col]].agg({'a':set,'b':set,'route_type':'first', catchment_col:'first'})
    link['node'] = link.apply(lambda row: row['a'].union(row['b']), axis=1)
    link = link.drop(columns=['a','b'])

    col_exist = col == 'route_type' # cannot explode if index == route_type (a column)
    link = link.explode('node').reset_index(drop=col_exist)
    link = node_dist.merge(link, left_on='node_index', right_on='node')
    #filter by distance
    link = link[link['distances'] <= link[catchment_col]]
    #drop duplicated mesh nodes (we count only one time)
    link = link.drop_duplicates(subset=['mesh_index',col],keep='first')

    return link.groupby(col)[pop_col].sum().to_dict()

# Catchment

In [35]:
assert densities != [], 'Please provide density columns in the zoning file'

meshes = {}
## road network here
for density in densities:
    if density == 'density' and 'density_population' not in densities:
        tag = 'population'
    elif density == 'density':
        tag = 'x'
    else:
        tag = density.split('density_')[1]

    print(tag)

    zonage[tag] = zonage[density] * zonage['area (km2)']

    if rnodes_file_provided and use_road_network:
        # use rnodes as mesh_pop.
        print('using road_nodes') ## road network here
        mesh = population_to_mesh(zonage, mesh=rnodes, step=step_size, col=tag, fill_missing='closest')
    else:
        # create a mesh
        #0.01 = 1km 0.005 = 500m
        mesh = population_to_mesh(zonage, mesh=None, step=step_size, col=tag, fill_missing='centroid')

    #mesh.to_file(output_folder + 'population_mesh.geojson',driver='GeoJSON',engine=io_engine)
    if catchment_radii_provided:
        max_dist = max([links[catchment_radius].max() for catchment_radius in catchment_radii])
    else:
        max_dist = max(max(input_catchment_radius.values()),default_catchment_radius)

    ## road network here, what to use if not road network?
    if rnodes_file_provided: 
        print('using road_nodes')
        node_dist = get_routing_distances(nodes, rnodes, rlinks, mesh, tag, 'length', max_dist)
    else:
        node_dist = get_acf_distances(nodes, mesh, tag, crs, max_dist)

    if catchment_radii_provided:
        for catchment_radius in catchment_radii:
            suf = catchment_radius.split('catchment_radius')[1]

            res_route = get_catchment_by_access('route_id', tag, catchment_radius, node_dist)
            res_mode = get_catchment_by_access('route_type', tag, catchment_radius, node_dist)

            if suf == '':
                df_route_id['catchment {}'.format(tag)] = res_route
                df_route_id['catchment {}'.format(tag)] = df_route_id['catchment {}'.format(tag)].fillna(0) 

                df_route_type['catchment {}'.format(tag)] = res_mode
                df_route_type['catchment {}'.format(tag)] = df_route_type['catchment {}'.format(tag)].fillna(0) 

            else:
                df_route_id['catchment {} {}'.format(tag, suf)] = res_route
                df_route_id['catchment {} {}'.format(tag, suf)] = df_route_id['catchment {} {}'.format(tag, suf)].fillna(0) 

                df_route_type['catchment {} {}'.format(tag, suf)] = res_mode
                df_route_type['catchment {} {}'.format(tag, suf)] = df_route_type['catchment {} {}'.format(tag, suf)].fillna(0) 

    else:
        res_route = get_catchment_by_mode('route_id', tag, node_dist)
        res_mode = get_catchment_by_mode('route_type', tag, node_dist)

        df_route_id['catchment {}'.format(tag)] = res_route
        df_route_id['catchment {}'.format(tag)] = df_route_id['catchment {}'.format(tag)].fillna(0) 

        df_route_type['catchment {}'.format(tag)] = res_mode
        df_route_type['catchment {}'.format(tag)] = df_route_type['catchment {}'.format(tag)].fillna(0)



population
using road_nodes
0 nodes in multiple zones. will be match to a single zone.
13 unfounded zones
using road_nodes
patate
using road_nodes
0 nodes in multiple zones. will be match to a single zone.
13 unfounded zones
using road_nodes


# Frequency

In [36]:
# Suppose that headway is the same in both directions : keep the minimum value
idx = df_route_id.groupby(level=0)['headway'].idxmin()
df_route_id = df_route_id.loc[idx]

df_route_id = df_route_id.rename(columns={'headway': 'headway (s)'})
df_route_id = df_route_id.sort_values('route_type', ascending=False)

In [37]:
links['frequency'] = 1/links['headway']

In [38]:
res = (links.groupby('route_id')['frequency'].agg('mean')*3600).to_dict()

df_route_id['frequency (veh/hours)'] = res
print(np.nansum([item for key,item in res.items()]))

18.0


In [39]:
res = (links.groupby('route_type')['frequency'].agg('mean')*3600).to_dict()

df_route_type['frequency (veh/hours)'] = res
print(sum([item for key,item in res.items()]))

6.0


In [40]:
link = (links.groupby(['route_id','trip_id'])[['frequency']].agg('mean')*3600)
res = link.reset_index().set_index('route_id')['frequency'].to_dict()
print(np.nansum([item for key,item in res.items()]))

18.0


In [41]:
link = (links.groupby(['route_type','trip_id'])[['frequency']].agg('mean')*3600)
res = link.reset_index().set_index('route_type')['frequency'].to_dict()
print(np.nansum([item for key,item in res.items()]))

6.0


# Operational Fleet

In [42]:
def get_fleet(col='route_id'):
    link = links.groupby([col,'trip_id'])[['time','frequency']].agg({'time':'mean','frequency':'mean'})
    link['fleet'] = np.ceil(link['frequency'] * link['time'])
    return link.reset_index().groupby(col)['fleet'].agg('sum').to_dict()


In [43]:
link = links.groupby(['route_id','trip_id'])[['time','frequency']].agg({'time':'sum','frequency':'mean'})
link['fleet'] = np.ceil(link['frequency'] * link['time'])
link
#return link.reset_index().groupby(col)['fleet'].agg('sum').to_dict()

time  frequency  fleet
route_id    trip_id                              
ChatelGuyon ChatelGuyon_0  4155   0.001667    7.0
            ChatelGuyon_1  4155   0.001667    7.0
Gannat      Gannat_0       3295   0.001667    6.0
            Gannat_1       3295   0.001667    6.0
Thiers      Thiers_0       7088   0.001667   12.0
            Thiers_1       7088   0.001667   12.0

In [44]:
res = get_fleet('route_id')

#df_route_id['fleet'] = res
#print(sum([item for key,item in res.items()]))
res

{'ChatelGuyon': 8.0, 'Gannat': 12.0, 'Thiers': 12.0}

In [45]:
links

,a,agency_id,b,direction_id,drop_off_type,headway,length,link_sequence,pickup_type,route_color,...,route_short_name,route_type,route_width,speed,time,trip_id,anchors,capacity,frequency,geometry
index,,,,,,,,,,,,,,,,,,,,,
link_qTNjaUDLm9x3yJrS9ubX9T,node_2zTroDrJuDCKmipdvjBrtL,QUENEDI,node_6CErzt5T6gFonN6RkpvAM6,0,0,600,18305,1,0,B5E0D6,...,Gannat,bus,3,20,3295,Gannat_0,None,60,0.001667,"LINESTRING (3.20420 46.09740, 3.20418 46.09763..."
link_wvPcELKEW52njEnuK7AeSe,node_f8Fb3oREhTRCmTVNW7TQzE,QUENEDI,node_b8e2FB6qm9PUQTLvQvNucR,0,0,600,6527,1,0,B5E0D6,...,ChatelGuyon,bus,3,20,1175,ChatelGuyon_0,"[ [ 3.0772146926584298, 45.914534641758706 ] ]",60,0.001667,"LINESTRING (3.05999 45.92018, 3.06060 45.92013..."
link_vTbX3V9qJJxThakJ1WZbbk,node_b8e2FB6qm9PUQTLvQvNucR,QUENEDI,node_wbHMs2n7zPVS1pp1cXnNw5,0,0,600,16554,2,0,B5E0D6,...,ChatelGuyon,bus,3,20,2980,ChatelGuyon_0,None,60,0.001667,"LINESTRING (3.07285 45.89581, 3.07269 45.89586..."
link_8EachMgBpjP7T2bWPiAc3E,node_4b2VnuC1zTsPzd9ueovVuH,QUENEDI,node_a3ZLvXzFnMqGNGu4hjEYT6,0,0,600,13020,1,0,B5E0D6,...,Thiers,bus,3,20,2344,Thiers_0,None,60,0.001667,"LINESTRING (3.50444 45.85987, 3.50422 45.85993..."
link_purLhqauca5SLLCFkShEC7,node_a3ZLvXzFnMqGNGu4hjEYT6,QUENEDI,node_rwwVbaGFJtCHoKTZ6B83Ax,0,0,600,26354,2,0,B5E0D6,...,Thiers,bus,3,20,4744,Thiers_0,None,60,0.001667,"LINESTRING (3.38307 45.84869, 3.38239 45.84873..."
link_4hsYkThjafc1NVpc8kLeid,node_rwwVbaGFJtCHoKTZ6B83Ax,QUENEDI,node_a3ZLvXzFnMqGNGu4hjEYT6,1,0,600,26354,1,0,B5E0D6,...,Thiers,bus,3,20,4744,Thiers_1,None,60,0.001667,"LINESTRING (3.09960 45.77888, 3.09940 45.77877..."
link_1ck6aSDmpEy7Jo2fbhNAxE,node_a3ZLvXzFnMqGNGu4hjEYT6,QUENEDI,node_4b2VnuC1zTsPzd9ueovVuH,1,0,600,13020,2,0,B5E0D6,...,Thiers,bus,3,20,2344,Thiers_1,None,60,0.001667,"LINESTRING (3.38307 45.84869, 3.38445 45.84860..."
link_2sfH8EPaWDjhzsZiZJMz1d,node_wbHMs2n7zPVS1pp1cXnNw5,QUENEDI,node_b8e2FB6qm9PUQTLvQvNucR,1,0,600,16554,1,0,B5E0D6,...,ChatelGuyon,bus,3,20,2980,ChatelGuyon_1,None,60,0.001667,"LINESTRING (3.10296 45.78059, 3.10359 45.78092..."
link_oCfgaw42pToYChhHThwCwy,node_b8e2FB6qm9PUQTLvQvNucR,QUENEDI,node_f8Fb3oREhTRCmTVNW7TQzE,1,0,600,6527,2,0,B5E0D6,...,ChatelGuyon,bus,3,20,1175,ChatelGuyon_1,"[ [ 3.0772146926584298, 45.914534641758706 ] ]",60,0.001667,"LINESTRING (3.07285 45.89581, 3.07269 45.89586..."


In [46]:
df_route_id

,route_type,veh_capacity (PAX),headway (s),catchment population,catchment patate,frequency (veh/hours)
route_id,,,,,,
ChatelGuyon,bus,60,600,2834.507547,8503.522642,6.0
Gannat,bus,60,600,2355.857359,7067.572077,6.0
Thiers,bus,60,600,2448.983014,7346.949042,6.0


In [47]:
res = get_fleet('route_type')

df_route_type['fleet'] = res
print(sum([item for key,item in res.items()]))

32.0


# Line Length

In [48]:
#def get_length(col='route_id',length_col='length'):
#    link = links.groupby([col,'trip_id'])[[length_col]].agg('sum')
#    return link.reset_index().groupby(col)[length_col].agg('sum').to_dict()   

In [49]:
def get_length(col='route_id', length_col='length'):
    link = links.groupby([col,'trip_id'])[[length_col]].agg(np.nansum)
    if col == 'route_type':
        return link.reset_index().groupby(col)[length_col].agg(np.nansum).to_dict()
    else:
        return link.reset_index().groupby(col)[length_col].agg(np.nanmean).to_dict()

In [50]:
# preparation. if length is NaN, or if shape dist travel exist.

length_col = None
if 'length' in links.columns and length_col == None:
    if len(links[links['length'].isnull()])==0:
        length_col = 'length'
        
if 'shape_dist_traveled' in links.columns and length_col == None:
    if len(links[links['shape_dist_traveled'].isnull()])==0:
        length_col = 'shape_dist_traveled'

if length_col == None:
    print('create length from geometry')
    links['length'] = links.to_crs(crs).length
    length_col = 'length'

In [51]:
res = get_length('route_id',length_col)

df_route_id['length (m)'] = res
print(sum([item for key,item in res.items()]))

80760.0


In [52]:
res = get_length('route_type',length_col)

df_route_type['length (m)'] = res
print(sum([item for key,item in res.items()]))

161520


# Number of station per line

In [53]:
# o-->o-->o-->o and  o<--o<--o<--o
# est-ce que j'ai 8 ou 4 stations ?
# j'ai 4 stations par trip et 4 stations par route (si c'est les memes).
# comment savoir si cest les memes. clustering?
# pour linstant. on prend tous les noeds unique par route_id ou route_type (col='route_id', route_id)
def get_num_station(col='route_id'):
    link = links.groupby(col)[['a','b']].agg({'a':set,'b':set})
    link['node_len'] = link.apply(lambda row: len(row['a'].union(row['b'])), axis=1)
    return link['node_len'].to_dict()


In [54]:
nodes['nindex'] = nodes.reset_index().index
nodes['stop_name'] = nodes.apply(lambda x: x['nindex'] if (pd.isna(x['stop_name']) or x['stop_name'] is None) else x['stop_name'], 1)
nodes.drop(columns='nindex', inplace=True)

In [55]:
dict_nb_trips = links[['route_id', 'trip_id']].drop_duplicates().groupby('route_id')['trip_id'].count().to_dict()
df_route_id['type'] = df_route_id.index.map(dict_nb_trips)
df_route_id['type'] = df_route_id['type'].apply(lambda x: 'circular' if x == 1 else 'linear')

In [56]:
def get_node_sequence(route_id):
    links_route = links.loc[links.route_id == route_id]
    if df_route_id.loc[route_id]['type'] == 'linear':
        trip_id = route_id + '_0'
    links_route = links_route.loc[links_route.trip_id == trip_id]
    links_route = links_route.sort_values(by='link_sequence')
    nodes_seq = []
    for i in range(len(links_route)):
        nodes_seq += [links_route.iloc[i]['a']]
    nodes_seq += [links_route.iloc[-1]['b']]
    return nodes_seq

In [57]:
nodes_stops = dict(zip(nodes.index, nodes['stop_name']))

def get_stops_sequence(route_id):
    nodes_seq = get_node_sequence(route_id)
    stops_seq = []
    for node in nodes_seq:
        stops_seq += [nodes_stops[node]]
    return stops_seq

In [58]:
df_route_id['stations sequence'] = [get_stops_sequence(route_id) for route_id in df_route_id.index]
df_route_id['nb stations'] = df_route_id['stations sequence'].apply(lambda x: len(x))

In [59]:
stations_route_type = pd.DataFrame(df_route_id.groupby('route_type')['stations sequence'].agg(lambda x: list(set(sum(x, [])))))
stations_route_type['nb stations'] = stations_route_type['stations sequence'].apply(lambda x: len(x))
df_route_type = df_route_type.merge(stations_route_type, left_on=df_route_type.index, right_on=stations_route_type.index, how='left')
df_route_type = df_route_type.rename(columns={'key_0': 'route_type'})
df_route_type = df_route_type.set_index('route_type')

## Connections

In [60]:
iterable = list(zip(nodes['stop_name'], nodes.index))

stops_nodes = defaultdict(set)
for key, value in iterable:
    stops_nodes[key].add(value)
stops_nodes = dict(stops_nodes)

In [61]:
iterable = list(zip(links['a'], links['route_id']))
iterable = iterable + list(zip(links['b'], links['route_id']))

nodes_routes = defaultdict(set)
for key, value in iterable:
    nodes_routes[key].add(value)
nodes_routes = dict(nodes_routes)

In [62]:
stops_routes = {}

for stop, node_list in stops_nodes.items():
    routes = set()
    for node in node_list:
        if node in nodes_routes:
            routes.update(nodes_routes[node])
    stops_routes[stop] = routes

In [63]:
hubs = pd.DataFrame.from_dict(stops_routes, orient='index')
hubs['lines'] = hubs.apply(lambda row: [val for val in row if pd.notnull(val)], axis=1)
hubs = hubs.drop(columns=[i for i in range(len(hubs.columns) - 1)])
hubs['nb_lines'] = hubs['lines'].apply(lambda x: len(x))
hubs = hubs.sort_values(by='nb_lines', ascending=False)

In [64]:
dict_route_type = dict(zip(df_route_id.index, df_route_id['route_type']))
dict_veh = dict(zip(df_route_id['route_type'], df_route_id['veh_capacity (PAX)']))
route_order = sorted(dict_veh, key=lambda x: int(dict_veh[x]), reverse=True)

def lines_to_dict(lines):
    route_dict = {route_type: [] for route_type in route_order}
    for line in lines:
        route_type = dict_route_type.get(line)
        if route_type in route_dict:
            route_dict[route_type].append(line)
    route_dict = {k: sorted(v) for k, v in route_dict.items() if v}
    return route_dict

hubs['lines'] = hubs['lines'].apply(lines_to_dict)

In [65]:
from shapely.ops import unary_union

def centroid(geometries):
    combined_geometry = unary_union(geometries)
    return combined_geometry.centroid

centroids = pd.DataFrame(nodes.groupby('stop_name')['geometry'].agg(centroid))
hubs = hubs.merge(centroids, left_on=hubs.index, right_on=centroids.index, how='left')

hubs = hubs.rename(columns={'key_0': 'stop_name'})
hubs = hubs.set_index('stop_name')

In [66]:
hubs['stop_radius'] = hubs['lines'].apply(lambda x: max(catchment_radius[mode] for mode in x.keys()))

In [67]:
def get_connections(row):
    route_id = row.name
    connections = set()
    for station in row['stations sequence']:
        if station in stops_routes:
            connections.update(stops_routes[station])
    connections.discard(route_id)  # Supprimer la route_id de l'ensemble des connexions
    return lines_to_dict(connections), len(connections)

df_route_id[['connexions', 'nb lines connected']] = df_route_id.apply(lambda row: pd.Series(get_connections(row)), axis=1)

# df_route_id[['connexions']].loc[df_route_id['connexions'] == 'tertiary']

In [68]:
# df_route_id

# Fleet

In [69]:
# if time_tabl

# Vehicle revenue KM 

In [70]:
def get_veh_kmh(col='route_id', length_col='length'):
    link = links.groupby([col,'trip_id'])[[length_col,'frequency']].agg({length_col:'sum','frequency':'mean'})
    link['veh_km/h'] = np.ceil(link['frequency'] * link[length_col]) * 3600/1000 #to km/H
    return link.reset_index().groupby(col)['veh_km/h'].agg('sum').to_dict()

In [71]:
res = get_veh_kmh('route_id')

df_route_id['veh.km/h'] = res
print(sum([item for key,item in res.items()]))

979.2


In [72]:
res = get_veh_kmh('route_type')

df_route_type['veh.km/h'] = res
print(sum([item for key,item in res.items()]))

979.2


# Round trip time

In [73]:
def get_round_trip_time(col='route_id'):
    link = links.groupby([col,'trip_id'])[['time']].agg('sum')
    return link.reset_index().groupby(col)['time'].agg('sum').to_dict()

In [74]:
res = get_round_trip_time('route_id')

df_route_id['round trip time (s)'] = res
# print(sum([item for key,item in res.items()]))

In [ ]:
# df_route_id

,route_type,veh_capacity (PAX),headway (s),catchment population,catchment patate,frequency (veh/hours),length (m),type,stations sequence,nb stations,connexions,nb lines connected,veh.km/h,round trip time (s)
route_id,,,,,,,,,,,,,,
ChatelGuyon,bus,60,600,2834.507547,8503.522642,6.0,23081.0,linear,"[Chatel Guyon, Riom, Clermont]",3,{'bus': ['Thiers']},1,280.8,8310
Gannat,bus,60,600,2355.857359,7067.572077,6.0,18305.0,linear,"[Gannat, Vichy]",2,{},0,223.2,6590
Thiers,bus,60,600,2448.983014,7346.949042,6.0,39374.0,linear,"[Thiers, Ornon, Clermont]",3,{'bus': ['ChatelGuyon']},1,475.2,14176


In [76]:
# df_route_type

In [77]:
# links

In [78]:
# stip

# export dfs to csv

In [82]:
# round numbers
#TODO : change label catchment
for col in ['catchment population', 'frequency (veh/hours)','length (m)','veh.km/h','round trip time (s)']:
    df_route_id[col] = df_route_id[col].apply(lambda x :np.round(x,2))
    df_route_id[col] = df_route_id[col].apply(lambda x :np.round(x,2))


In [83]:
#df_route_id = df_route_id.fillna('null')
#df_route_type = df_route_type.fillna('null')

In [84]:
df_route_id.to_csv(output_folder+'route_id_metrics.csv')
# df_route_id

In [85]:
df_route_type.to_csv(output_folder+'route_type_metrics.csv')
# df_route_type

In [86]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined

# Formation:

exporter une image (.png) <br>
ex: barplot du catchement pour les route_type

In [87]:
plot = df_route_type.reset_index().plot(kind='bar', x='route_type', y='catchment', color='#559bb4', rot=0, figsize=[10, 5])
plot.set_title('Couverture population par mode')
plot.set_ylabel('')
plot.set_xlabel("route_type")
plot.legend([])

KeyError: 'catchment'

# geomatic outputs

In [ ]:
#using get catchment. get the catchment radius of each node (get larger one if used by many mode.)
link = links.groupby('route_type')[['a','b','route_type']].agg({'a':set,'b':set,'route_type':'first'})
link['node'] = link.apply(lambda row: row['a'].union(row['b']), axis=1)
link = link.drop(columns=['a','b'])
# add catchment radius for the route_type
link['catchment_radius'] = link['route_type'].apply(lambda x: catchment_radius.get(x,default_catchment_radius))
link = link.explode('node').reset_index(drop=True)
link = link.sort_values('catchment_radius',ascending=False).drop_duplicates('node',keep='first')
link = node_dist.merge(link, left_on='node_index', right_on='node')
link = link[link['distances'] <= link['catchment_radius']]

temp_dict = link.groupby('node_index')['population'].sum().to_dict()
nodes['catchment'] = nodes.index.map(temp_dict.get)

temp_dict = link.groupby('node_index')['catchment_radius'].agg('first').to_dict() 
nodes['catchment_radius'] = nodes.index.map(temp_dict.get)


In [ ]:
nodes.to_file(output_folder+'nodes.geojson',driver='GeoJSON',engine=io_engine)

# test